In [ ]:
import pandas as pd
from sklearn.cluster import KMeans 
import matplotlib.pyplot  as plt
import seaborn as sns

In [ ]:
 df = pd.read_csv('E:/incheon/parcel_20210105_58131.csv', encoding='UTF-8', sep="|", error_bad_lines=False)

In [ ]:
df=df.dropna(subset=['위도'])
df=df.dropna(subset=['경도'])
df = df.reset_index(drop=True)

In [ ]:
feature = df[ ['위도','경도']]
feature.head()

In [ ]:

# k-means clustering 실행
kmeans = KMeans(n_clusters=30)
kmeans.fit(feature)

# 결과 확인
r = feature.copy()
r["predict"] = kmeans.labels_
r.head()

In [ ]:
r.head()

In [ ]:
#pairplot with Seaborn
plt.rcParams['font.family'] = 'NanumGothic'
sns.pairplot(r,hue='predict')
plt.show()

In [ ]:
# 0.0001
r['위도'] = r['위도'].round(4)
r['경도'] = r['경도'].round(4)
GPS_point=r.groupby('predict')[['위도','경도']].agg( lambda x: pd.Series.mode(x)[0]).reset_index() ## 평균 GPS

In [ ]:
GPS_point.head()

In [ ]:
sum(r.groupby('predict').count()['위도']>200)

In [ ]:
r.groupby('predict')[['위도','경도']].agg(pd.Series.count).reset_index() ## 평균 GPS

In [ ]:
GPS_point=GPS_point[GPS_point['predict']<sum(r.groupby('predict').count()['위도']>100)]

In [ ]:
#사용 라이브러리
import numpy as np
import pandas as pd
from tqdm import tqdm
import folium
from folium.plugins import MarkerCluster
from urllib.request import Request, urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from lxml import html
import time
import requests, bs4
from urllib.parse import urlencode, quote_plus, unquote
from bs4 import BeautifulSoup
import json
pd.set_option('display.max_columns', 500)

In [ ]:
lat = GPS_point['위도'].mean()
long = GPS_point['경도'].mean()
m = folium.Map([lat,long],zoom_start=10)

In [ ]:

for i in GPS_point.index:
    sub_lat =  GPS_point.loc[i,'위도']
    sub_long = GPS_point.loc[i,'경도']
    title = GPS_point.loc[i,'predict']            
    #지도에 데이터 찍어서 보여주기
    folium.Marker([sub_lat,sub_long],icon=folium.Icon(color='red', icon='home', prefix='fa'),tooltip=title).add_to(m)
    folium.Circle([sub_lat,sub_long],tooltip=title,radius=2500).add_to(m)

#한글이 안나오는 오류로 html로 trouble shooting 
m

In [ ]:
def plot(df):
    # 리스트를 이용해 여러 행의 데이터를 위,경도로 묶음
    locations = list(zip(df.위도, df.경도))
    # 반복문을 이용해 locations로 정의한 변수에 해당하는 위치에 자동차 모양의 빨간색 아이콘을 지정했다.
    icons = [folium.Icon(icon="box", prefix="fa", color="green") for _ in range(len(locations))]


    # 클러스터를 생성하여 앞에서 지정한 locations, icons을 설정해준다.
    cluster = MarkerCluster(locations = locations, icons = icons)
    # 지도에 클러스터를 추가.
    cluster.add_to(m)

In [ ]:
plot(df)
m

In [ ]:
#  m.save('E:/kmeans30.html')

In [ ]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [ ]:
GPS_point['좌표']=GPS_point['경도'].astype(str)+','+GPS_point['위도'].astype(str)

In [ ]:
GPS_point.head()

In [ ]:
GPS_point.loc[5,'위도']

In [ ]:
res=[]
x=[]
for j in tqdm(df.index):
    lat =  df.loc[j,'위도']
    long = df.loc[j,'경도']    

    min_dist = 50
    for i in GPS_point.index:
        sub_lat =  GPS_point.loc[i,'위도']
        sub_long = GPS_point.loc[i,'경도']
        min_dist=min(min_dist,GeoUtil.get_harversion_distance(sub_long, sub_lat,long, lat))
    x.append(min_dist)
    base=np.array(x)

In [ ]:
print('평균 지선 거리',base.mean().round(2),'km')
print('지선 거리 총합',base.sum().round(2),'km')

In [ ]:
#네이버 경로 탐지 및 도로 시각화 코드
def path_geocord(gps1, gps2, color):
    client_id = 'xhbfckkyg3';    # 본인이 할당받은 ID 입력
    client_pw = 'lpd28lbrhk1WqcLPSmEzUUSAL0mrM5lwyMCAZoaz';    # 본인이 할당받은 Secret 입력
    api_url = 'https://naveropenapi.apigw.gov-ntruss.com/map-direction/v1/driving?start='
    url_mid = '&goal='
    path_coordi = []
    url = api_url + gps1 + url_mid + gps2
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['code'] != 0:
                print(response_body["message"])
                distance=0
                duration=None
                path=None
            else:
                print(response_body["message"])
                distance = response_body["route"]['traoptimal'][0]['summary']['distance']
                duration = response_body["route"]['traoptimal'][0]['summary']['duration']
                path = response_body["route"]['traoptimal'][0]['path']
    path_coordi.append([distance, duration, path])
    np_path_coordi = np.array(path_coordi)
    pd_path_coordi = pd.DataFrame({
                              "이동거리": np_path_coordi[:, 0],
                              "소요시간": np_path_coordi[:, 1],
                              "전체이동경로": np_path_coordi[:, 2]})
    points=[]
    for j in range(len(pd_path_coordi['전체이동경로'][0])):
        points.append(sorted(pd_path_coordi['전체이동경로'][0][j]))
    folium.PolyLine(points,tooltip = i,color=color).add_to(m)
    print("해당 지선 이동거리:", pd_path_coordi['이동거리'][0]/1000, "km")
    print("해당 지선 이동시간:", pd_path_coordi['소요시간'][0]/60000, "분")

In [ ]:
path_geocord(GPS_point['좌표'][6],GPS_point['좌표'][18],'red')

In [ ]:
basecamp_GPS = pd.read_csv('E:/post/basecamp_geo.csv', encoding='UTF-8', sep="|", error_bad_lines=False)

In [ ]:
basecamp_GPS.head()

In [ ]:
path_geocord(basecamp_GPS['GPS'][1],GPS_point['좌표'][18],'blue')

In [ ]:
post = pd.read_csv('E:/post/picmt0060.csv', encoding='UTF-8', sep=",", error_bad_lines=False)
subpost=post[post['POADDR'].str.contains('인천')]
subpost = subpost.iloc[:,[0,16,18,19,65,66]]

In [ ]:
subpost.head()

In [ ]:
subpost=subpost.dropna(subset=['RDGISX'])
subpost=subpost.dropna(subset=['RDGISY'])
subpost['좌표']=subpost['RDGISX'].astype(str)+','+subpost['RDGISY'].astype(str)

In [ ]:
subpost.reset_index(drop=True, inplace=True)

In [ ]:
subpost['좌표'][121]

In [ ]:
path_geocord(subpost['좌표'][121],GPS_point['좌표'][5],'green')

In [ ]:
m